In [1]:
import pandas as pd
import numpy as np

In [2]:
data_dir = 'C:/ZhangLI/Codes/DataSet/爱奇艺用户留存预测/'
user_port = pd.read_csv(data_dir+'user_portrait_data.csv')  # 40M

In [10]:
app_launch = pd.read_csv(data_dir+'app_launch_logs.csv')  # 194M

In [14]:
video_related = pd.read_csv(data_dir + 'video_related_data.csv')  # 162M

In [18]:
user_playback = pd.read_csv(data_dir + 'user_playback_data.csv')  # 2168M

In [21]:
user_interaction = pd.read_csv(data_dir + 'user_interaction_data.csv')

In [19]:
user_port.memory_usage().sum()/ (1024**2)  
app_launch.memory_usage().sum()/ (1024**2)  
video_related.memory_usage().sum()/ (1024**2)  
user_playback.memory_usage().sum()/ (1024**2)  

2168.153045654297

In [15]:
app_launch.user_id.value_counts()

10561857    128
10431750    114
10341025     98
10045632     97
10542336     95
           ... 
10112125      1
10175284      1
10355201      1
10320586      1
10181348      1
Name: user_id, Length: 600000, dtype: int64

In [20]:
user_playback  # 用户观看视频 的信息  
# 用户的留存   比如 131登录 在 132出现的概率

,user_id,item_id,playtime,date
0,10057286,20628283.0,2208.612,145
1,10522615,23930557.0,31.054,145
2,10494028,20173699.0,115.952,145
3,10181987,21350426.0,1.585,145
4,10439175,22946929.0,51.726,145
...,...,...,...,...
71046030,10485189,21266571.0,1.780,153
71046031,10065355,20649533.0,110.807,153
71046032,10453102,23966955.0,2590.954,153
71046033,10029701,22403648.0,113.477,153


In [22]:
user_interaction  # 数据压缩， 数据对其

,user_id,item_id,interact_type,date
0,10243056,22635954,1,213
1,10203565,24723827,3,213
2,10317559,20413036,2,213
3,10158940,23833050,1,213
4,10376271,22218154,1,213
...,...,...,...,...
198603,10407802,22901903,10,181
198604,10407802,22901903,10,181
198605,10407802,22901903,10,181
198606,10407802,20797334,7,181


In [32]:
app_launch.date.max() # 220
app_launch.date.min() # 100
launch_grp = app_launch.groupby('user_id').agg(launch_date=('date',list), launch_type=('launch_type', list)).reset_index()

In [47]:
# 为每个用户生成一个样本，
test_launch = launch_grp.loc[0:3, :]
def get_d(list_date):
    n1, n2 = min(list_date), max(list_date)
    # print(n1, n2)
    if n1 < n2-7:
        end_date = np.random.randint(n1, n2 - 7)
    else:
        end_date = np.random.randint(100, 222 - 7)
    return end_date
    
launch_grp['end_date'] = launch_grp.launch_date.apply(get_d)

def get_label(row):
    launch_list = row.launch_date
    end = row.end_date
    # print(end, set(launch_list))
    label = sum([1 for x in set(launch_list) if end < x < end+8])
    return label
launch_grp["label"] = launch_grp.apply(get_label, axis=1)

test_launch  # 构架标签的方式 也是上分的点， 这个方法可以增加数据量
launch_grp.label.value_counts()

train = launch_grp

In [51]:
#test_without_label.csv
#submit_example.csv
# 提交 
train = launch_grp[["user_id", "end_date", "label"]]
train
test = pd.read_csv(data_dir + 'test_without_label.csv')
test['label'] = -1
# 提交表格
# test = pd.read_csv(data_dir + 'submit_example.csv')
data = pd.concat([train, test], ignore_index=True)
data

,user_id,end_date,label
0,10000000,169,0
1,10000001,142,0
2,10000002,150,0
3,10000003,181,0
4,10000004,103,0
...,...,...,...
614996,10355586,205,-1
614997,10589773,210,-1
614998,10181954,218,-1
614999,10544736,164,-1


In [56]:
# 登录数据的处理
# new_test = test.merge(launch_grp[["user_id", "launch_type", "launch_date"]], how="left", on="user_id")
launch_grp = launch_grp.append(
    test.merge(launch_grp[["user_id", "launch_type", "launch_date"]], how="left", on="user_id")
)  # ignore_index=True default=False
launch_grp

,user_id,launch_date,launch_type,end_date,label
0,10000000,"[195, 192, 196, 193, 191, 141, 187, 202, 132, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",169,0
1,10000001,"[118, 117, 114]","[0, 0, 0]",142,0
2,10000002,"[129, 144, 128, 140, 163, 160]","[0, 0, 0, 0, 0, 0]",150,0
3,10000003,"[144, 144]","[1, 0]",181,0
4,10000004,"[157, 154, 155, 156]","[0, 0, 0, 0]",103,0
...,...,...,...,...,...
14996,10355586,"[118, 115, 124]","[0, 1, 0]",205,-1
14997,10589773,"[183, 144, 173, 167, 203, 204, 178, 155, 177, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",210,-1
14998,10181954,"[152, 155, 153, 144, 150, 143, 158, 151, 154, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",218,-1
14999,10544736,"[138, 117, 105, 148, 113, 116, 115, 158, 154, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",164,-1


In [ ]:
# 获取最后31天的登录方式
launch_grp.launch_date

In [73]:
# 获取最近31天的登录情况， 这个写的很巧妙， 首先对两个进行排序，然后groupby取最大，之后再进行操作
def gen_launch_seq(row):
    seq_sort = sorted(zip(row.launch_type, row.launch_date), key=lambda x: x[1])
    seq_map = {k: max(g)[0] + 1 for k, g in groupby(seq_sort, lambda x: x[1])}
    end = row.end_date
    seq = [seq_map.get(x, 0) for x in range(end-31, end+1)]
    return seq
launch_grp["launch_seq"] = launch_grp.apply(gen_launch_seq, axis=1)
launch_grp

# 最后只要了 launch_seq

,user_id,launch_date,launch_type,end_date,label,launch_seq
0,10000000,"[195, 192, 196, 193, 191, 141, 187, 202, 132, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",169,0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,10000001,"[118, 117, 114]","[0, 0, 0]",142,0,"[0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."
2,10000002,"[129, 144, 128, 140, 163, 160]","[0, 0, 0, 0, 0, 0]",150,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, ..."
3,10000003,"[144, 144]","[1, 0]",181,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,10000004,"[157, 154, 155, 156]","[0, 0, 0, 0]",103,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...
14996,10355586,"[118, 115, 124]","[0, 1, 0]",205,-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
14997,10589773,"[183, 144, 173, 167, 203, 204, 178, 155, 177, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",210,-1,"[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, ..."
14998,10181954,"[152, 155, 153, 144, 150, 143, 158, 151, 154, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",218,-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
14999,10544736,"[138, 117, 105, 148, 113, 116, 115, 158, 154, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",164,-1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [70]:
# payback date and vedio data process

[(0, 23), (1, 24), (1, 35), (0, 35), (0, 56)]